## 提示学习

提示学习（Prompt Learning）是一种在自然语言处理（NLP）中使用的技术，通过为模型提供简短的提示（prompt）或模板来指导其生成特定类型的输出。这种方法可以被视为一种“少参数化”的方式，相对于传统的基于大型预训练模型的方法，它所需的参数较少。

提示学习的原理是利用简短的提示文本来指导模型生成特定类型的文本。这些提示文本通常包含关键词或结构，可以帮助模型更好地理解任务要求。通过在输入中添加提示，模型可以更容易地学习到与任务相关的知识，从而提高其性能。

举一个中文例子，假设我们有一个情感分类任务，要求将一段中文文本分为积极、中性、消极三类。传统的方法可能是使用一个大型的预训练语言模型，如BERT或GPT，来直接进行分类。而在提示学习中，我们可以为模型提供以下提示：“这是一段积极/中性/消极的评论：”，然后将待分类的文本填入提示中，模型只需根据填入的文本生成对应的分类结果。

少样本提示学习（Few-Shot Prompt Learning）和零样本提示学习（Zero-Shot Prompt Learning）是提示学习的两种变体：
- 少样本提示学习：模型只能从少量样本中学习，通常使用的样本数量较少，但模型可以通过这些样本来生成符合预期的结果。
- 零样本提示学习：模型可以在没有任何样本的情况下学习，并生成符合预期的输出。这通常通过在提示中提供足够的信息来实现。

提示学习解决了传统方法中的一些问题，包括：
- 数据效率：在少样本或零样本的情况下，提示学习可以达到相似甚至更好的性能。
- 灵活性：可以根据任务的不同需求进行定制，更灵活地适应各种应用场景。
- 解释性：由于提示文本通常比较简单明了，因此生成的结果更容易解释和理解。

提示学习最早由OpenAI在2020年提出，并在其GPT-3模型中得到了广泛的应用。

In [1]:
from transformers import AutoModelForMaskedLM , AutoTokenizer
import torch
model_path=r"E:\code\bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

d:\ProgramData\Anaconda3\envs\torch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load Prompting class
from prompt import Prompting
prompting= Prompting(model=model_path)

Some weights of the model checkpoint at E:\code\bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
prompt="Because it was [MASK]." #提示模板
text="I really like the film a lot." #输入文本

In [4]:
prompting.prompt_pred(text+prompt)[:10] #语言模型生成的概率最高的10个词，大都是正向评价

[('fun', tensor(10.2700)),
 ('funny', tensor(9.7243)),
 ('amazing', tensor(9.6926)),
 ('good', tensor(9.6492)),
 ('awesome', tensor(9.5985)),
 ('great', tensor(9.4881)),
 ('awful', tensor(8.8970)),
 ('interesting', tensor(8.7762)),
 ('beautiful', tensor(8.4037)),
 ('incredible', tensor(8.3900))]

In [4]:
text="I did not like the film."
prompting.prompt_pred(prompt+text)[:10] #语言模型生成的概率最高的10个词，大都是负向评价

[('awful', tensor(8.9539)),
 ('funny', tensor(8.2918)),
 ('horrible', tensor(7.7996)),
 ('terrible', tensor(7.7877)),
 ('bad', tensor(7.6291)),
 ('true', tensor(7.5571)),
 ('me', tensor(7.5084)),
 ('scary', tensor(7.4047)),
 ('stupid', tensor(7.3304)),
 ('ridiculous', tensor(7.2885))]

In [5]:
text="not worth watching"
#直接根据语言模型预测词的概率来判断输入语句的类别，达到不需要任务样本进行训练的效果
prompting.compute_tokens_prob(text+prompt, token_list1=["great","amazin","good"], token_list2= ["bad","awfull","terrible"])

tensor([0.6934, 0.3066])

In [1]:
from transformers import AutoModelForMaskedLM , AutoTokenizer
import torch
model_path=r"E:\code\ernie-3.0-base-zh"
tokenizer = AutoTokenizer.from_pretrained(model_path)

d:\ProgramData\Anaconda3\envs\torch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load Prompting class
from prompt import Prompting
prompting= Prompting(model=model_path)

Some weights of the model checkpoint at E:\code\ernie-3.0-base-zh were not used when initializing ErnieForMaskedLM: ['ernie.pooler.dense.bias', 'ernie.pooler.dense.weight']
- This IS expected if you are initializing ErnieForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ErnieForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
prompt="这是一条垃圾短信吗？答案是[MASK]"

In [4]:
text="感谢致电杭州萧山全金釜韩国烧烤店，本店位于金城路xxx号。韩式烧烤等，价格实惠、欢迎惠顾【全金釜韩国烧烤店】"
#tensor([0.9505, 0.0495])

In [5]:
text="我是不明真相的萌萌哒吃瓜群众"
#tensor([0.3495, 0.6505])

In [6]:
prompting.prompt_pred(text+prompt)[:10]

[('？', tensor(15.0272)),
 ('这', tensor(12.0955)),
 ('！', tensor(11.8858)),
 ('：', tensor(11.7694)),
 ('辣', tensor(10.7749)),
 ('yes', tensor(10.6435)),
 ('呢', tensor(10.4846)),
 ('。', tensor(10.4053)),
 ('…', tensor(10.3930)),
 ('那', tensor(10.2409))]

In [7]:
prob = prompting.compute_tokens_prob(text+prompt, token_list1=["是", "真"], token_list2= ["否", "假"])
print(prob)

tensor([0.9505, 0.0495])


In [8]:
prob = prompting.compute_tokens_prob(prompt, token_list1=["是", "真"], token_list2= ["否", "假"])
print(prob)

tensor([0.5654, 0.4346])


In [9]:
import pandas as pd
import numpy as np

In [10]:
#1、加载数据
train_df = pd.read_csv(r'data.csv', encoding='utf-8', header=None, names=['label','review'])
print(train_df.shape)

sentences = list(train_df['review'][1:])
label =train_df['label'][1:].values

(1243, 2)


In [11]:
labels = []
for s in label:
    labels.append(int(s))

In [14]:
pre = []
for text in sentences[:100]:
    prob = prompting.compute_tokens_prob(text+prompt, token_list1=["是", "真"], token_list2= ["否", "假"])
    if prob[0] > 0.5:
        pre.append(0)
    else:
        pre.append(1)

In [15]:
from sklearn import metrics
classify_report = metrics.classification_report(labels[:100], pre, digits = 4) #分类报告 support测试集样本数
print(classify_report) 
confusion_matrix = metrics.confusion_matrix(labels[:100], pre) #混淆矩阵
print(confusion_matrix) 

              precision    recall  f1-score   support

           0     0.8046    0.8861    0.8434        79
           1     0.3077    0.1905    0.2353        21

    accuracy                         0.7400       100
   macro avg     0.5561    0.5383    0.5393       100
weighted avg     0.7002    0.7400    0.7157       100

[[70  9]
 [17  4]]
